In [1]:
import constants
from constants import *
import importlib
importlib.reload(constants)

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

from matplotlib import pyplot as plt
import pandas as pd

import time

import sklearn
import tensorflow as tf
import numpy as np
import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *

#from sklearn.metrics import *
#from sklearn.metrics import classification_report
# import seaborn as sns

import sys


## Generate tensorflow dataset
Si hay clases muy desbalanceadas, para balancearlas:

Repetir imagenes aleatorias hasta completar las que faltan para que esté balanceado -> se retrasa el salvado de la red porque los callback no se actualizan hasta que acaba una epoch
Elegir subconjuntos de imagenes aleatorias por cada clase del tamaño de la clase menor y darselas a la red e ir salvandola (porque la epoch ha terminado) por cada subconjunto que se le pasa

## Generación del dataset

In [2]:
# get_min(dataframe)
# given a dataframe returns the value of instances of the class with fewer instances (minimum class)
def get_min(dataframe):

    min_class = list(dataframe['class'].value_counts())[-1]

    return min_class


# create_pd_dataframe(data_csv, csv_folder = CSV_FOLDER)
# given the name of the csv and the folder where it is located, create a pandas dataframe with its content
# and modify the class from string to number with the class dictionary in constants
def create_pd_dataframes(data_csv, csv_folder = CSV_FOLDER):
    data_panda = pd.read_csv(os.path.join(csv_folder, data_csv))

    # Añadir el path absoluto (la parte desde C:// ... hasta animals al inicio para poder acceder
    data_panda["path"] = [DATASETS_ROOT_FOLDER + item for item in data_panda["path"]]

    data_panda["class"] = [CLASS_DICTIONARY[str(item)] for item in data_panda["class"]]

    '''
    num_min_instances = get_min(data_panda)
    classes = list(np.unique(data_panda['class']))

    new_dataframe = pd.DataFrame([])

    # new_dataframe.columns = ["class", "path"] #set the header row as the df header

    for clas in classes:
        instances_of_class = data_panda.loc[data_panda["class"] == clas]
        instances_of_class.sample(n = num_min_instances, replace = False)
        new_dataframe = pd.concat([new_dataframe,instances_of_class])
    '''
    return data_panda



# balance_df_classes(dataframe, mini = 0)
# given a dataframe and the minimum name of instances we want per classs, it create a balaced dataframe
def balance_df_classes(dataframe, mini = 0):
    new_dataframe = pd.DataFrame([])

    min_class = get_min(dataframe)

    # To add a minimum number of images per class in the case of entering it by parameters
    if mini > min_class:
        min_class = mini

    # for each class take the min number of imgs that must be taken to create the dataframe
    for clas in list(np.unique(dataframe['class'])):
        current_dataframe = (dataframe[dataframe['class'] == clas])

        aux = current_dataframe.sample(n = min_class, replace = True)

        # y lo concatena con el que ya teniamos
        new_dataframe = pd.concat([new_dataframe,aux])

    return new_dataframe

# create_balanced_dataframes(data_csv, csv_folder = CSV_FOLDER, mini = 0)
# calls the functions that create the dataframes and balance them
def create_balanced_dataframes(data_csv, csv_folder = CSV_FOLDER, mini = 0):

    pandas_dataframe = create_pd_dataframes(data_csv, csv_folder)
    pandas_dataframe_balanced = balance_df_classes(pandas_dataframe, mini)

    return pandas_dataframe_balanced

## Pasar de un dataframe a un dataset de tensorflow

In [3]:
# create_tf_ds(dataframe)
# from a dataframe, it creates a tensorflow dataset
def create_tf_ds(dataframe):
    datagen = tf.keras.preprocessing.image.ImageDataGenerator(
                                  rescale=1./255.
                                  )

    tf_dataset = datagen.flow_from_dataframe(
                            dataframe,
                            x_col = 'path',
                            y_col = 'class',
                            target_size = (IMG_WIDTH,IMG_HEIGHT),
                            color_mode = 'rgb',
                            batch_size = 32,
                            shuffle = True,
                            interpolation = "bicubic",
                            class_mode='raw' # categorical -> str , raw = numbers
                            )

    return tf_dataset

## Modelo

In [4]:
MODEL_NAME = "test_model"
FROM_LOGITS = True
MODEL_PATH = None

In [5]:
# Creates the model
'''if MODEL_PATH:
    model = tf.keras.models.load_model(MODEL_PATH)
else:
    model = tf.keras.models.Sequential([
      tf.keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH, N_CHANNELS)),
      tf.keras.layers.MaxPooling2D((2, 2)),
      tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
      tf.keras.layers.MaxPooling2D((2, 2)),
      tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(CLASS_NUMBER)
    ])'''

model = tf.keras.models.Sequential()
model.add(Conv2D(32,kernel_size=5,padding='same',activation='relu',
        input_shape=(28,28,3)))
model.add(MaxPool2D(padding='same'))
model.add(Conv2D(64,kernel_size=5,padding='same',activation='relu'))
model.add(MaxPool2D(padding='same'))
model.add(Conv2D(128,kernel_size=3,padding='same',activation='relu'))
model.add(MaxPool2D(padding='same'))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(CLASS_NUMBER))

In [6]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 32)        2432      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 14, 14, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 14, 14, 64)        51264     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 7, 7, 64)         0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 7, 7, 128)         73856     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 4, 4, 128)        0

## Hiperparametros del entramiento

In [7]:
# callback to save the best model

# save best model in validation
# monitoriza el accuracy (mejor de validation)
# solo salva el mejor a true
# no solo salva los pesos, salva modelo + pesos en el mismo file
# en validation se salvará el max
# con freq cada epoch
callback_1 = tf.keras.callbacks.ModelCheckpoint(
os.path.join(MODEL_SAVE_FOLDER,MODEL_NAME), monitor='val_sparse_categorical_accuracy', verbose=0, save_best_only=True,
save_weights_only=False, mode='max', save_freq='epoch'
)

# callback to stop the model if it hasnt improved in 50 epochs
callback_2 = tf.keras.callbacks.EarlyStopping(
monitor='val_sparse_categorical_accuracy', min_delta=0.001, patience=50, verbose=0, mode='max',
baseline=None, restore_best_weights=True
)


## Define Parameters (Loss, Accuracy and Optimizer)

In [8]:
# A loss object that receives the raw network output and the one-hot raw class labels is created
# sparse porque le entran los numeros en vez del vector de onehot
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=FROM_LOGITS)

# Una funcion Loss funcionaría:
# Loss(,)
# la red devuelve un batch con vectores = y (puede ser como logits o softmax), por otro lado están las clases reales: un batch de números representando la clase o un vector de onehot.
# le entra un vector de la salida de la red (bacth), y el numero )
# en un loss binario ln(0.5) = 0.69314718056 -> es aleatorio, mala interpretacion


# a metric object that calculates the accuracy is created
accuracy = tf.keras.metrics.SparseCategoricalAccuracy()

# optimizer object is created
# it was used adam
optimizer = SGD(learning_rate=0.01)

## Compile Model

In [9]:
# model compilation
model.compile(optimizer=optimizer, loss=loss, metrics=accuracy)
# model.compile(optimizer="adam", loss=loss, metrics=["accuracy"])

## Bucle de entrenamiento

In [14]:
for current_epoch in range(EPOCHS):
    # TO-DO Poner bonito el bucle para que quede clara la informacion, que no print validated image filenames


    ######################## DATASET GENERATION ########################
    # only when we start a new training loop dataset renewal, else we would only train with the dataset during 1 epoch
    if current_epoch % TRAINING_LOOPS_DATASETS_RENEWAL == 0:
        print("\n==========================================================\n")
        print("Generating new dataset")
        # Pandas dataframe generation from csvs and balanced
        train_df = create_balanced_dataframes(TRAIN_CSV, CSV_FOLDER)
        val_df = create_balanced_dataframes(VALIDATION_CSV, CSV_FOLDER, mini = 30)

        # Tensorflow dataframe generation
        train_tf_dataset = create_tf_ds(train_df)
        validation_tf_dataset = create_tf_ds(val_df)

        print("\n")

    ######################## TRAINING PHASE ########################
    print("Training with created dataset")
    # h = model.fit(train_tf_dataset, epochs=1, validation_data = validation_tf_dataset, batch_size = BATCH_NUMBER, callbacks= [callback_1, callback_2], verbose = 1)
    # print("\n==========================================================\n")



Generating new dataset
Found 1257 validated image filenames.
Found 237 validated image filenames.


Training with created dataset
Training with created dataset
Training with created dataset


Generating new dataset
Found 1257 validated image filenames.
Found 237 validated image filenames.


Training with created dataset
Training with created dataset
Training with created dataset


Generating new dataset
Found 1257 validated image filenames.
Found 237 validated image filenames.


Training with created dataset
Training with created dataset
Training with created dataset


In [ ]:
# In case it is decided to train more than 100 epochs
for i in range(epochs):

    with tf.device('/device:GPU:0'):
    #SECUENCIA PARA CONFECCIÓN DEL CSV

    history['epoch'] = list(range(0,len(history.index)))

    history.to_csv(os.path.join(CSV_FOLDER, name_csv_history), header=True, index=False)

    # Plot history: CE
    plt.figure()
    plt.plot(history['epoch'], history['loss'], label='Loss (training data)')
    plt.plot(history['epoch'], history['val_loss'], label='Loss (validation data)')
    plt.title('Loss')
    plt.ylabel('CE')
    plt.xlabel('Nº epoch')
    plt.legend(loc="upper left")
    plt.show()

    # Plot history: Accuracy
    plt.figure()
    plt.plot(history['epoch'], history['categorical_accuracy'], label='Accuracy (training data)')
    plt.plot(history['epoch'], history['val_categorical_accuracy'], label='Accuracy (validation data)')
    plt.title('Accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Nº epoch')
    plt.legend(loc="upper left")
    plt.show()

    # show training time
    fin = time.time()
    print(round((fin - inicio)/60, 0))

In [ ]:
# In case it is decided to train more than 100 epochs
for i in range(epochs):

    with tf.device('/device:GPU:0'):
    #SECUENCIA PARA CONFECCIÓN DEL CSV

    history['epoch'] = list(range(0,len(history.index)))

    history.to_csv(os.path.join(CSV_FOLDER, name_csv_history), header=True, index=False)

    # Plot history: CE
    plt.figure()
    plt.plot(history['epoch'], history['loss'], label='Loss (training data)')
    plt.plot(history['epoch'], history['val_loss'], label='Loss (validation data)')
    plt.title('Loss')
    plt.ylabel('CE')
    plt.xlabel('Nº epoch')
    plt.legend(loc="upper left")
    plt.show()

    # Plot history: Accuracy
    plt.figure()
    plt.plot(history['epoch'], history['categorical_accuracy'], label='Accuracy (training data)')
    plt.plot(history['epoch'], history['val_categorical_accuracy'], label='Accuracy (validation data)')
    plt.title('Accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Nº epoch')
    plt.legend(loc="upper left")
    plt.show()

    # show training time
    fin = time.time()
    print(round((fin - inicio)/60, 0))